In [1]:
using JuMP
using BilevelOptimization

┌ Info: Precompiling BilevelOptimization [98803d92-2a2a-5e5c-9642-fb423c87776e]
└ @ Base loading.jl:1260


In [2]:
# problem taken from:
# A.G. Mersha, S. Dempe,
# Linear bilevel programming with upper-level constraints depending on lower-level solution.
# Appl. Math. Computation (180), 2006
cx = [-1.]
cy = [-2.]
G = zeros(2,1) .+ [-2.,1.]
H = zeros(2,1) .+ [3.,1.]
q = [12.,14.]
d = [-1.]
A = zeros(2,1) .+ [-3.,3.]
B = ones(2,1)
b = [-3.,30.]

2-element Array{Float64,1}:
 -3.0
 30.0

In [3]:
prob = BilevelLP(
        cx, cy,
        G, H, q,
        d, A, B, b
    )

BilevelLP{Array{Float64,1},Array{Float64,2},Array{Float64,2},Array{Bool,1}}([-1.0], [-2.0], [-2.0; 1.0], [3.0; 1.0], [12.0, 14.0], [-1.0], [-3.0; 3.0], [1.0; 1.0], [-3.0, 30.0], 1, 1, 2, 2, [0.0], [Inf], Bool[1], [Inf], Int64[], [0.0])

In [5]:
using Cbc

┌ Info: Precompiling Cbc [9961bab8-2fa3-5c5a-9d89-47fab24efd76]
└ @ Base loading.jl:1260


In [6]:
(m, x, y, λ, _) = build_blp_model(prob, optimizer_with_attributes(Cbc.Optimizer, "LogLevel" => 0))

(A JuMP Model
Minimization problem with:
Variables: 7
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 3 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 2 constraints
`Array{VariableRef,1}`-in-`MathOptInterface.SOS1{Float64}`: 3 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 7 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)
Names registered in the model: lowercons, nonnegative_sos_constraints, rhs_sos_constraints, s, uppercons, x, y, λ, σ, VariableRef[x[1]], VariableRef[y[1]], VariableRef[λ[1], λ[2]], VariableRef[s[1], s[2]])

In [7]:
JuMP.optimize!(m)

Cbc3007W No integer variables - nothing to do
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -LogLevel 0 -solve -quit (default strategy 1)


In [8]:
# Printing the optimal solutions obtained
println("Optimal Solutions:")
println("x1 = ",JuMP.value(x[1]))
println("y1 = ",JuMP.value(y[1]))


Optimal Solutions:
x1 = 8.0
y1 = 5.999999999999999


In [9]:
# Example 2
# problem taken from:
# Bilevel Programming Problems
# S. Dempe, V. Kalashnikov, G. Pérez-Valdés, N. Kalashnykova
# Springer 2015
cx = [2.,1.]
cy = [2.,-1.]
G = zeros(0,2)
H = zeros(0,2)
q = Float64[]
d = [0.,0.]
F = [1. 0.;0. 1.]
A = zeros(1,2)
B = [-2. 1.]
b = [0.]

1-element Array{Float64,1}:
 0.0

In [10]:
prob = BilevelLP(
        cx, cy,
        G, H, q,
        d, A, B, b, Int[], F
    )

BilevelLP{Array{Float64,1},Array{Float64,2},Array{Float64,2},Array{Bool,1}}([2.0, 1.0], [2.0, -1.0], Array{Float64}(undef,0,2), Array{Float64}(undef,0,2), Float64[], [0.0, 0.0], [0.0 0.0], [-2.0 1.0], [0.0], 2, 2, 0, 1, [0.0, 0.0], [Inf, Inf], Bool[1, 1], [Inf, Inf], Int64[], [1.0 0.0; 0.0 1.0])

In [11]:
set_lower_bound(prob, BilevelOptimization.upper, 1, -1.)
setupperbound(prob, BilevelOptimization.upper, 1, 1.)
set_lower_bound(prob, BilevelOptimization.upper, 2, -1.)
setupperbound(prob, BilevelOptimization.upper, 2, -0.75)
set_lower_bound(prob, BilevelOptimization.lower, 1, -Inf64)
setupperbound(prob, BilevelOptimization.lower, 1, 2.)
set_lower_bound(prob, BilevelOptimization.lower, 2, 0.)
setupperbound(prob, BilevelOptimization.lower, 2, 2.)

3

In [12]:
(m, x, y, λ, _) = build_blp_model(prob, optimizer_with_attributes(Cbc.Optimizer, "LogLevel" => 0))

(A JuMP Model
Minimization problem with:
Variables: 12
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 6 constraints
`Array{VariableRef,1}`-in-`MathOptInterface.SOS1{Float64}`: 5 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 11 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 2 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)
Names registered in the model: lowercons, nonnegative_sos_constraints, rhs_sos_constraints, s, uppercons, x, y, λ, σ, VariableRef[x[1], x[2]], VariableRef[y[1], y[2]], VariableRef[λ[1], λ[2], λ[3]], VariableRef[s[1], s[2], s[3]])

In [13]:
 JuMP.optimize!(m)

Cbc3007W No integer variables - nothing to do
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -LogLevel 0 -solve -quit (default strategy 1)
3 relaxed row infeasibilities - summing to 4
3 relaxed row infeasibilities - summing to 4
3 relaxed row infeasibilities - summing to 4


In [14]:
# Printing the optimal solutions obtained
println("Optimal Solutions:")
println("x1 = ",JuMP.value(x[1]))
println("x2 = ",JuMP.value(x[2]))

Optimal Solutions:
x1 = -1.0
x2 = -1.0
